# Handling Missing Data with Pandas
pandas borrows all the capabilities from numpy selection + adds a number of convenient methods to handle missing values. Let's see one at a time:

## Hands on!

In [3]:
import numpy as np
import pandas as pd

## Pandas utility functions
Similarly to `numpy`, pandas also has a few utility functions to identify and detect null values:

In [5]:
pd.isnull(np.nan)

True

In [6]:
pd.isnull(None)

True

In [7]:
pd.isna(np.nan)

True

In [8]:
pd.isna(None)

True

The opposite ones also exist:

In [10]:
pd.notnull(None)

False

In [11]:
pd.notnull(np.nan)

False

In [12]:
pd.notna(np.nan)

False

In [13]:
pd.notnull(3)

True

These functions also work with Series and DataFrames:

In [15]:
pd.isnull(pd.Series([1,np.nan,7]))

0    False
1     True
2    False
dtype: bool

In [16]:
pd.notnull(pd.Series([1,np.nan,7]))

0     True
1    False
2     True
dtype: bool

In [17]:
pd.isnull(pd.DataFrame({
    'Column A' : [1, np.nan, 7],
    'Column B' : [np.nan,2,3],
    'Column C' : [np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


## Pandas Operations with Missing Values
Pandas manages missing values more gracefully than numpy. `nan`s will no longer behave as "viruses", and operations will just ignore them completely:

In [19]:
pd.Series([1,2,np.nan]).count()

2

Because `.count()` in pandas counts only the non-missing values (i.e., values that are not NaN).

In [21]:
pd.Series([1,2,np.nan]).sum()

3.0

The sum() method in pandas automatically ignores NaN values.

It adds only the valid numbers: 1 + 2 = 3.0

The result is float because np.nan is a float, so pandas promotes the data type.

In [23]:
pd.Series([2, 2, np.nan]).mean()

2.0

mean() calculates the average of non-NaN values only by default.

In [2, 2, np.nan], the two valid numbers are 2 and 2.

So the mean is:
(
2
+
2
)
/2=
2.0
(2+2)/2=2.0

Want to include NaN in the mean?
You’d have to fill or handle them manually using `.fillna()` before calling `.mean()`.


## Filtering missing data
As we saw with numpy, we could combine boolean selection + pd.isnull to filter out those nans and null values:

In [27]:
s = pd.Series([1,2,3,np.nan,np.nan, 4])

In [28]:
pd.notnull(s)

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [29]:
pd.isnull(s)

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [30]:
pd.isnull(s).sum()

2

In [31]:
pd.notnull(s).sum()

4

In [32]:
s[pd.notnull(s)]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

Uses that boolean mask to select only the True values:

But both `notnull` and `isnull` are also methods of Series and DataFrames, so we could use it that way:

In [35]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [36]:
s.notnull()

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [37]:
s[s.notnull()]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

## Dropping null values
Boolean selection + `notnull()` seems a little bit verbose and repetitive. And as we said before: any repetitive task will probably have a better, more DRY way. In this case, we can use the `dropna` method:

In [39]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

In [40]:
s.dropna()

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

## Dropping null values on DataFrames
You saw how simple it is to drop nas with a Series. But with `DataFrames`, there will be a few more things to consider, because you can't drop single values. You can only drop entire columns or rows. Let's start with a sample `DataFrame`:

In [42]:
df = pd.DataFrame({
    "Column A" : [1, np.nan, 30, np.nan],
    "Column B" : [2,8,31,np.nan],
    "Column C" : [np.nan, 9,32,100],
    "Column D" : [5,8,34,110]
})

In [43]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [44]:
df.shape

(4, 4)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 260.0 bytes


In [46]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [47]:
df.isnull().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

In [48]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34



In this case we're dropping <b>rows</b>. Rows containing null values are dropped from the DF. You can also use the `axis` parameter to drop columns containing null values:

In [50]:
df.dropna(axis=1)  # axis='columns' also works

,Column D
0,5
1,8
2,34
3,110


In [51]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [52]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [53]:
df.dropna(thresh=4)

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


dropna() removes rows (by default) that contain missing values (NaN).

The thresh= parameter sets a threshold for how many non-NaN values a row must have to be kept.

In [55]:
df.dropna(thresh =3, axis ='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


## Filling null values
Sometimes instead than dropping the null values, we might need to replace them with some other value. This highly depends on your context and the dataset you're currently working. Sometimes a `nan` can be replaced with a `0`, sometimes it can be replaced with the `mean` of the sample, and some other times you can take the closest value. Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem.

In [57]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

<b>Filling nulls with a arbitrary value</b>

In [59]:
s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    4.0
dtype: float64

In [60]:
s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    2.5
4    2.5
5    4.0
dtype: float64

In [61]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

### Filling nulls with contiguous (close) values

The method argument is used to fill null values with other values close to that null one:

In [63]:
s.ffill()

0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    4.0
dtype: float64

Fills NaN with the last known non-NaN value (from above).

In [65]:
s.bfill()

0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
5    4.0
dtype: float64

Fills NaN with the next non-NaN value (from below).

<b>This can still leave null values at the extremes of the Series/DataFrame:</b>

In [68]:
n=pd.Series([np.nan, 3, np.nan, 9])

In [69]:
n.ffill()

0    NaN
1    3.0
2    3.0
3    9.0
dtype: float64

In [70]:
m = pd.Series([1, np.nan, 3, np.nan, np.nan])
m

0    1.0
1    NaN
2    3.0
3    NaN
4    NaN
dtype: float64

In [71]:
m.bfill()

0    1.0
1    3.0
2    3.0
3    NaN
4    NaN
dtype: float64

## Filling null values on DataFrames
The fillna method also works on DataFrames, and it works similarly. The main differences are that you can specify the axis (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [73]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [74]:
df.fillna(0)

,Column A,Column B,Column C,Column D
0,1.0,2.0,0.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,0.0,100.0,110


In [75]:
df.fillna({'Column A' :0, 'Column B' : 99, 'Column C' : df['Column C'].mean()})

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [76]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [77]:
df.ffill(axis=0)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [78]:
df.ffill(axis=1)

,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


## Checking if there are NAs
The question is: Does this `Series` or `DataFrame` contain any missing value? The answer should be yes or no: `True` or `False`. How can you verify it?

<b>Example 1: Checking the length</b>

If there are missing values, `s.dropna()` will have less elements than `s`:

In [80]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

In [81]:
s.dropna().count()

4

In [82]:
len(s)

6

In [83]:
len(s.dropna())

4

In [84]:
missing_Values = len(s.dropna()) !=len(s)
missing_Values

True

There's also a count method, that excludes nans from its result:


<b>More Pythonic solution `any`</b>

The methods `any` and `all` check if either there's `any` True value in a Series or `all` the values are `True`. They work in the same way as in Python:

In [87]:
pd.Series([True, False, False]).any()

True

In [88]:
pd.Series([True, False, False]).all()

False

The `isnull()` method returned a Boolean `Series` with `True` values wherever there was a `nan`:

In [90]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

So we can just use the any method with the boolean array returned:

In [92]:
pd.Series([1, np.nan]).isnull()

0    False
1     True
dtype: bool

In [93]:
pd.Series([1, np.nan]).any()

True

In [94]:
pd.Series([1, np.nan]).isnull().any()

True

In [95]:
s.isnull().any()

True

A more strict version would check only the `values` of the Series:

In [97]:
s.isnull().values

array([False, False, False,  True,  True, False])

In [98]:
s.isnull().values.any()

True

In [173]:
import pandas as pd
import numpy as np

s = pd.Series(['a', 3, np.nan, 1, np.nan])

print(s.notnull().sum())

3
